In [1]:
import os

In [10]:
%pwd

'g:\\End-to-End-ML-Project\\End-to-end-Ml-project'

In [4]:
os.chdir("../")

In [6]:
from dataclasses import dataclass
from pathlib import Path

@dataclass (frozen = True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [8]:
from mlproject.constants import *
from mlproject.utils.common import read_yaml, create_directories


In [34]:
# creating class Configuration Manager for data Ingestion with making directiories,
#reading yaml files
class ConfigurationManager:
    #constants dir contains path of config, paarams, schema
    def __init__(self, config_pathfile = CONFIG_PATH_FILE, schema_pathfile = SCHEMA_PATH_FILE, 
                 params_pathfile = PARMAS_PATH_FILE):      
            
        # read_yaml function read the yaml file & return it into dictionary using ConfigBox
        self.config = read_yaml(config_pathfile) 
        self.params = read_yaml(params_pathfile)
        self.schema = read_yaml(schema_pathfile)

        # create_directories will create directory inside config.yaml under key artifacts_root
        create_directories([self.config.artifacts_root])
    
    # function get_data_ingestion_config will values under key data_ingestion
    # & create directiory & also return the values in function DataIngestionConfig
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])
        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL= config.source_URL,
            local_data_file= config.local_data_file,
            unzip_dir= config.unzip_dir
        )
        return data_ingestion_config

In [35]:
import zipfile
import os
import urllib.request as request
from mlproject import logger
from mlproject.utils.common import get_size

In [40]:
# constructing class DataIngestion for downloading the data & unzipping the file
class DataIngestion:
    # constructor containg config variable with class name DataIngestionConfig
    def __init__(self, config: DataIngestionConfig):
        self.config = config

    # function download_file will download the file from url
    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, header = request.urlretrieve(url= self.config.source_URL, filename=self.config.local_data_file)
            logger.info("f{filename} download! with following file info: \n{header}")
        else:
            logger.info(f"File already exists of size:{get_size(Path(self.config.local_data_file))}")
        
    # function extract_zip_file will extract zip file into directory
    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(file=self.config.local_data_file, mode='r') as zip_ref:
            zip_ref.extractall(unzip_path)

In [41]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config = data_ingestion_config) 
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e



[2024-01-07 22:01:34,322: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-01-07 22:01:34,331: INFO: common: yaml file: params.yaml loaded successfully]
[2024-01-07 22:01:34,351: INFO: common: yaml file: schema.yaml loaded successfully]
[2024-01-07 22:01:34,355: INFO: common: created directory at: artifacts]
[2024-01-07 22:01:34,359: INFO: common: created directory at: artifacts/data_ingestion]
[2024-01-07 22:01:34,363: INFO: 3531090047: File already exists of size:~ 24 KB]
